# Mediapipe

In [3]:
import cv2
import mediapipe as mp
import openpifpaf
import PIL
import torch
from time import time

In [4]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA GeForce RTX 4070 Ti


In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

In [6]:
# For static images:
IMAGE_FILES = []
BG_COLOR = (192, 192, 192) # gray
with mp_holistic.Holistic(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    refine_face_landmarks=True) as holistic:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    image_height, image_width, _ = image.shape
    # Convert the BGR image to RGB before processing.
    results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if results.pose_landmarks:
      print(
          f'Nose coordinates: ('
          f'{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width}, '
          f'{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y * image_height})'
      )

    annotated_image = image.copy()
    # Draw segmentation on the image.
    # To improve segmentation around boundaries, consider applying a joint
    # bilateral filter to "results.segmentation_mask" with "image".
    condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
    bg_image = np.zeros(image.shape, dtype=np.uint8)
    bg_image[:] = BG_COLOR
    annotated_image = np.where(condition, annotated_image, bg_image)
    # Draw pose, left and right hands, and face landmarks on the image.
    mp_drawing.draw_landmarks(
        annotated_image,
        results.face_landmarks,
        mp_holistic.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_tesselation_style())
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.
        get_default_pose_landmarks_style())
    cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)
    # Plot pose world landmarks.
    mp_drawing.plot_landmarks(
        results.pose_world_landmarks, mp_holistic.POSE_CONNECTIONS)

# Using the webcam

In [12]:
predictor = openpifpaf.Predictor(checkpoint='shufflenetv2k16')

pos_dict = {
        "nose": 0,
        "left_eye": 1,
        "right_eye": 2,
        "left_ear": 3,
        "right_ear": 4,
        "left_shoudler": 5,
        "right_shoudler": 6,
        "left_elbow": 7,
        "right_elbow": 8,
        "left_wrist": 9,
        "right_wrist": 10,
        "left_hip": 11,
        "right_hip": 12,
        "left_knee": 13,
        "right_knee": 14,
        "left_ankle": 15,
        "right_ankle": 16}

RED = (0, 0, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)
YELLOW = (0, 255, 255)

In [13]:
OPPF = False

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    start = time()
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'co£ntinue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    if OPPF:
        #OpenPifPaf
        pil_img = PIL.Image.fromarray(image)
        predictions, _, _ = predictor.pil_image(pil_img)

        try:
            
            #draw arms and hips
            keypoints = predictions[0].data

            # left arm
            left_shoulder = keypoints[pos_dict["left_shoudler"],0:2]
            left_hip = keypoints[pos_dict["left_hip"],0:2]
            left_elbow = keypoints[pos_dict["left_elbow"],0:2]

            # right arm
            right_shoulder = keypoints[pos_dict["right_shoudler"],0:2]
            right_hip = keypoints[pos_dict["right_hip"],0:2]
            right_elbow = keypoints[pos_dict["right_elbow"],0:2]

            cv2.line(image, tuple(map(int, tuple(left_shoulder))), tuple(map(int, tuple(left_elbow))), RED, 5)
            cv2.line(image, tuple(map(int, tuple(left_shoulder))), tuple(map(int, tuple(left_hip))), YELLOW, 5)
            cv2.line(image, tuple(map(int, tuple(right_shoulder))), tuple(map(int, tuple(right_elbow))), RED, 5)
            cv2.line(image, tuple(map(int, tuple(right_shoulder))), tuple(map(int, tuple(right_hip))), YELLOW, 5)
        except:
            cv2.putText(img = image, text=f"no detection", org = (0,60), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=GREEN,thickness=2)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))
    c = cv2.waitKey(1)
    if c == 27: # press escape to quit
        break
    stop = time()

    # writes fps
    fps = 1/(stop-start)
    cv2.putText(img = image, text=f"{fps:.2f} fps", org = (0,30), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=5, color=GREEN,thickness=2)

cap.release()
cv2.destroyAllWindows()
